In [129]:
import os
import yfinance as yf
import spacy
import bs4 as bs
import requests
from collections import Counter
from string import punctuation
import json

In [175]:
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})
tickers = []
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)
tickers = [s.replace('\n', '') for s in tickers]
print(len(tickers))

505


In [ ]:
infos = {}
for i in tickers:
    print(i)
    ticker = yf.Ticker(i)
    infos[i] = ticker.info
with open("sp500infos.json", "w") as outfile:
    json.dump(infos, outfile)

In [244]:
sp500data = []
with open("sp500infos.json", "r") as inputfile:
    sp500data = json.load(inputfile)
sp500dict = dict(sp500data)

In [303]:
with open('common_words.txt') as f:
    lines = f.readlines()
common_words = [i.replace('\n','') for i in lines ]

In [245]:
sp500dict['AMD']['longName']
commas = 'Advanced Micro Devices, Inc.'.replace(',', '')
dots = commas.replace('.', '')
split = dots.split(' ')
remove_end = ' '.join(split[0:-1])
remove_end

'Advanced Micro Devices'

In [284]:
longnames = {}
shortnames = {}
tickernames = {}
for i in tickers:
    ticker = sp500dict[i]
    longnames[ticker['longName']] = i
    shortnames[ticker['shortName']] = i
    tickernames[i]=i
dictionary = {**longnames, **shortnames, **tickernames}
del dictionary[None]
del longnames[None]

True

In [274]:
with open("sp500tickers.json", "w") as outfile:
    json.dump(tickernames, outfile)

In [275]:
count_dict = {}
longnames_low = dict([(i.lower(), longnames[i] ) for i in longnames ])
for i in longnames_low:
    split = [j for j in i.split(' ') if j != '']
    for j in split:
        if j not in count_dict:
            count_dict[j] = 1
        else:
            count_dict[j] = count_dict[j] + 1
not_unique = [i for i in count_dict if count_dict[i] > 1]
unique = [i for i in count_dict if count_dict[i] == 1]
print(not_unique)

['company', 'laboratories', 'inc.', 'plc', 'devices,', 'the', 'corporation', 'incorporated', 'technologies,', 'air', 'and', 'group,', 'technology,', 'energy', 'american', 'airlines', 'group', 'electric', 'power', 'company,', 'international', '(reit)', 'works', 'financial,', 'materials,', 'networks,', 'j.', '&', 'co.', 'communities,', 'bank', 'of', 'america', 'co.,', 'laboratories,', 'holdings', 'boston', 'properties,', 'scientific', 'financial', 'solutions,', 'systems,', 'entertainment,', 'capital', 'global', 'energy,', 'industries', 'holdings,', 'charles', 'international,', 'limited', 'technology', 'brands,', 'companies,', 'corp.', 'health', 'digital', 'realty', 'trust,', 'services', 'dollar', 'general', 'dominion', 'duke', 'resources,', 'property', 'ltd.', 'storage', 'trust', 'national', 'services,', 'bancorp', 'republic', 'brands', 'home', 'henry', 'enterprise', 'worldwide', 'resorts,', 'huntington', 'industries,', 'johnson', 'southern', 'martin', 'n.v.', 'marathon', 'petroleum', 'b

In [276]:
######UNIQUE NAMES########
unique_short_names = {}
for i in unique:
    for j in longnames_low:
        if i in j:
            unique_short_names[i] = longnames_low[j]
            break
dictionary = {**dictionary, **unique_short_names}

In [277]:
######LONG NAMES WITHOUT COMMON WORDS#####
longnames_unique = {}
for i in longnames_low:
    current_name = i
    for j in not_unique:
        current_name = current_name.replace(j, '')
    current_name = current_name.replace(',', ' ')
    current_name = current_name.replace('.', ' ')
    current_name = current_name.replace('-', ' ')
    current_name = current_name.strip()
    current_name = ' '.join([j for j in current_name.split(' ') if j != ''])
    if current_name != '' and len(current_name) > 3:
        longnames_unique[current_name] = longnames_low[i]
dictionary = {**dictionary, **longnames_unique}

In [278]:
### MIGHT BE UNNECESARRY####
ends_removed_dict = {}
for i in dictionary:
    temp = i.replace(',', ' ')
    temp = temp.replace('.', ' ')
    temp = temp.replace('-', ' ')
    temp = temp.strip()
    split = [j for j in temp.split(' ') if j != '']
    remove_end = ' '.join(split[0:-1])
    if remove_end not in dictionary:
        ends_removed_dict[remove_end] = dictionary[i]
dictionary = {**dictionary, **ends_removed_dict}

In [279]:
lowercase = dict([(i.lower(), dictionary[i] ) for i in dictionary ])
with open("sp500.json", "w") as outfile:
    json.dump(lowercase, outfile)

In [306]:
names = {**longnames_unique}
remove_words = []
for i in names:
    if i in common_words:
        remove_words.append(i)
for i in remove_words:
    del names[i]
with open("sp500longnames.json", "w") as outfile:
    json.dump(names, outfile)

In [293]:
def matcher(company, lookup_dictionary):
        current_name = company.strip().lower()
        current_name = current_name.replace(',', ' ')
        current_name = current_name.replace('.', ' ')
        current_name = current_name.replace('-', ' ')
        split = [j for j in current_name.split(' ')]
        full_name = ' '.join(split)
        first_name = split[0]
        if full_name in lookup_dictionary:
            return lookup_dictionary[full_name]
        elif first_name in lookup_dictionary:
            return lookup_dictionary[first_name]

In [295]:
print(matcher('Bank of America', lowercase)) 
print(matcher('Amazon', lowercase)) 
print(matcher('Advanced micro devices', lowercase)) 
print(matcher('Disney', lowercase)) 
print(matcher('WMT\n\n\n\n\n\n', lowercase)) 
print(matcher('Microsoft Corp', lowercase)) 
print(matcher('mcdonald s', lowercase)) 
print(matcher('micron technology inc', lowercase)) 
print(matcher('Freeport-McMoran Copper', lowercase)) 

BAC
AMZN
AMD
DIS
WMT
MSFT
None
MU
FCX


True

In [189]:
list_dict = [(i,longnames_good[i]) for i in longnames_good ]
good = []
for i in range(len(list_dict)):
    is_good = True
    current, _  = list_dict[i]
    for j in range(0, len(list_dict)):
        if i == j:
            continue
        compare, _  = list_dict[j]
        if current in compare:
            
            print( str(i) +  ': ' + current + '  '+ str(j) +  ': ' + compare )
            is_good = False
            break
    if is_good:
        good.append(list_dict[i])

24: lines  121: delta  lines
26: tower  421: welltower
27: water  419: waters
95: citi  96: citizens
126: discover  127: discovery
129: tree  368: state street
137: edison  106: consolidated edison
205: idex  206: idexx
216: inter  213: intercontinental exchange
218: intuit  219: intuitive surgical
283: morgan  229: jpmorgan chase
329: pool  425: whirlpool
335: service  360: servicenow
364: sourn  231: kansas city sourn
422: pharmaceutical  346: regeneron pharmaceuticals
423: rock  58: blackrock
426: williams  361: sherwinwilliams


In [162]:
print(str(len(list_dict)) + ' ' + str(len(good))  )

437 420


In [ ]:
longnames_good = dict(good)
longnames_good

In [308]:
names['news']

KeyError: 'news'

In [307]:
[i for i in lowercase if lowercase[i] == 'APC']

[]

In [309]:
tickers

['MMM',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADBE',
 'AMD',
 'AAP',
 'AES',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'ANTM',
 'AON',
 'AOS',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BLL',
 'BAC',
 'BBWI',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'CHRW',
 'CDNS',
 'CZR',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CDAY',
 'CERN',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMC